In [3]:
%matplotlib inline

# Author: Xiang Wang

from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms, models
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion()   # interactive mode

## 一、文件夹结构设计和数据存放

### 1. 建立存放数据的文件夹

In [49]:
# shutil －一种高层次的文件操作工具，强大之处是在对文件的复制与删除操作支持比较好
import os
import shutil

In [ ]:
# 建立data文件夹用于存放数据，train和val分别存放训练集和验证集
# .mkdir的函数用于建立该路径下的文件夹
os.mkdir('/Users/macbook/Documents/Programme/pytorch/Data/CIFAR-10/Data')
os.mkdir('/Users/macbook/Documents/Programme/pytorch/Data/CIFAR-10/Data/train')
os.mkdir('/Users/macbook/Documents/Programme/pytorch/Data/CIFAR-10/Data/val')

In [ ]:
# 得到id和label，以便以下面分割数据集
import pandas as pd

idLabel = pd.read_csv('/Users/macbook/Documents/Programme/pytorch/Data/CIFAR-10/trainLabels.csv')

In [ ]:
# 创建训练集分类的label的文件夹
train_root = '/Users/macbook/Documents/Programme/pytorch/Data/CIFAR-10/Data/train'

for label in idLabel.label.unique():
    os.mkdir(os.path.join(train_root, label))

In [ ]:
# 创建测试集分类的label的文件夹
val_root = '/Users/macbook/Documents/Programme/pytorch/Data/CIFAR-10/Data/val'

for label in idLabel.label.unique():
    os.mkdir(os.path.join(val_root, label))

### 2. 分割数据集并将其存入对应文件夹


In [ ]:
# .listdir主要用于列出该路径下所有的文件的名称，以列表的形式得到
# 这里的路径是下载下来的CIFAR-10的训练集数据存放的路径
data_file = os.listdir('/Users/macbook/Documents/Programme/pytorch/Data/CIFAR-10/zip/train')

In [ ]:
# 分割数据集
import numpy as np
idLabel_train = np.array(idLabel[['id','label']])[:48000]
idLabel_val = np.array(idLabel[['id','label']])[48000:]

In [ ]:
# 移动图片在训练集的相应位置
root = '/Users/macbook/Documents/Programme/pytorch/Data/CIFAR-10/'

for (id,label) in idLabel_train:
    
    pic_path = root + 'zip/train/' + str(id) + '.png'
    
    obj_path = train_root + '/' + str(label) + '/' + str(id) + '.png'
    
    shutil.copy(pic_path, obj_path)

In [ ]:
# 移动图片到验证集的相应位置
root = '/Users/macbook/Documents/Programme/pytorch/Data/CIFAR-10/'

for (id,label) in idLabel_val:
    
    pic_path = root + 'zip/train/' + str(id) + '.png'
    
    obj_path = val_root + '/' + str(label) + '/' + str(id) + '.png'
    
    shutil.copy(pic_path, obj_path)

## 二、数据处理与加载

In [19]:
print('==> Preparing data..')

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
    ]),

    'val': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
    ]),
    'test': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
    ]),
}


==> Preparing data..


In [20]:
data_dir = '/Users/macbook/Documents/Programme/pytorch/Data/CIFAR-10/Data/'

image_datasets = {x: torchvision.datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) 
                  for x in ['train', 'val', 'test']}

# 得到可batch的数据格式
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=1)
               for x in ['train', 'val']}

testdataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=False, num_workers=1)
                   for x in ['test']}


# 得到数据规模
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val','test']}

# 得到分类名
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## 三、建立模型

In [8]:
def conv3x3(in_channel, out_channel, stride=1):
    return nn.Conv2d(in_channel, out_channel, 3, stride=stride, padding=1, bias=False)

In [7]:
# 首先定义residual块，这个模块的作用是定义一个正常的卷积块之外，将这个模块的输入直接加到输出
# 这个过程中由于可能会出现输入输出通道不相同的情况，在模块中用if分别对应两种情况
# input：num_convs：卷积的次数
#       in_channels：输入的通道数
#       out_channels：输出的通道数
#
# output：返回residual块对象
class residual_block(nn.Module):
    def __init__(self, in_channel, out_channel, same_shape=True):
        super(residual_block, self).__init__()
        self.same_shape = same_shape
        stride=1 if self.same_shape else 2
        
        self.conv1 = conv3x3(in_channel, out_channel, stride=stride)
        self.bn1 = nn.BatchNorm2d(out_channel)
        
        self.conv2 = conv3x3(out_channel, out_channel)
        self.bn2 = nn.BatchNorm2d(out_channel)
        if not self.same_shape:
            self.conv3 = nn.Conv2d(in_channel, out_channel, 1, stride=stride)
        
    def forward(self, x):
        out = self.conv1(x)
        out = F.relu(self.bn1(out), True)
        out = self.conv2(out)
        out = F.relu(self.bn2(out), True)
        
        if not self.same_shape:
            x = self.conv3(x)
        return F.relu(x+out, True)

In [9]:
class resNet(nn.Module):
    def __init__(self, in_channel, num_classes, verbose=False):
        super(resNet, self).__init__()
        self.verbose = verbose
        
        self.block1 = nn.Conv2d(in_channel, 64, 7, 2)
        
        self.block2 = nn.Sequential(
            nn.MaxPool2d(3, 2),
            residual_block(64, 64),
            residual_block(64, 64)
        )
        
        self.block3 = nn.Sequential(
            residual_block(64, 128, False),
            residual_block(128, 128)
        )
        
        self.block4 = nn.Sequential(
            residual_block(128, 256, False),
            residual_block(256, 256)
        )
        
        self.block5 = nn.Sequential(
            residual_block(256, 512, False),
            residual_block(512, 512),
            nn.AvgPool2d(3)
        )
        
        self.fc = nn.Linear(512, num_classes)
        
    def forward(self, x):
        x = self.block1(x)
        if self.verbose:
            print('block 1 output: {}'.format(x.shape))
        x = self.block2(x)
        if self.verbose:
            print('block 2 output: {}'.format(x.shape))
        x = self.block3(x)
        if self.verbose:
            print('block 3 output: {}'.format(x.shape))
        x = self.block4(x)
        if self.verbose:
            print('block 4 output: {}'.format(x.shape))
        x = self.block5(x)
        if self.verbose:
            print('block 5 output: {}'.format(x.shape))
        x = x.view(x.shape[0], -1)
        x = self.fc(x)
        
        return x


In [11]:
model_ft = resNet(3,10)

In [12]:
model_ft

resNet(
  (block1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2))
  (block2): Sequential(
    (0): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (1): residual_block(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (2): residual_block(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (block3): S

## 四、设置必要的参数

In [21]:
# 设置为GPU处理
model_ft = model_ft.to(device)

# 设置损失函数
criterion = nn.CrossEntropyLoss()
# 设置优化算法
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

## 五、训练阶段

In [22]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    
    # static_dict()用于返回包含模块所有状态的字典，包括参数和缓存。键是参数名称或者缓存名称。
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # 加入一个batch，将得到的数据放置到制定的设备上（GPU或者CPU）
            for inputs, labels in dataloaders[phase]:
                inputs = inputs
                labels = labels

                # 清空梯度
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                # 以前是使用.data[0]来得到loss，不过官方推荐使用item来转化这里数据
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [4]:
# model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,num_epochs=2)